# Overview

This notebook will pull data for each player for each game as much as possible as well as the outcome of each game, then output the file to a feather file so it's prepared for more ML work.

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)

# Step 1: Pull Player Stats

In [ ]:
import stats

players = stats.get_player_season_stats(year=2023, season_type="regular")

# Step 2: Pull Draft Pick States for that Year

In [ ]:
import draft

draft = draft.get_draft_picks(year=2024)
draft.info()

# Step 3: Merge it together

In [ ]:
import pandas as pd
import numpy as np


def merge_player_draft_data(players, draft):
    """
    Merge player statistics with draft data, ensuring all stat categories
    exist for each player with zeros for missing values.

    Parameters:
    players (DataFrame): Player statistics with columns including playerId, category, statType, stat
    draft (DataFrame): Draft data with collegeAthleteId

    Returns:
    DataFrame: Draft data with player statistics added as columns
    """
    # Step 1: Get all unique stat combinations
    stat_combinations = players[["category", "statType"]].drop_duplicates()
    print(f"Found {len(stat_combinations)} unique category-statType combinations")

    # Ensure consistent types for both IDs before comparison
    # Convert playerId to numeric type if it's not already
    if players["playerId"].dtype != "int64":
        players["playerId"] = pd.to_numeric(players["playerId"], errors="coerce")
        # Drop rows with NaN playerId
        players = players.dropna(subset=["playerId"])
        # Convert to int64
        players["playerId"] = players["playerId"].astype("int64")

    # For draft data, handle NaN values differently - we'll keep them
    # Convert collegeAthleteId to numeric type if it's not already
    if draft["collegeAthleteId"].dtype != "int64":
        draft["collegeAthleteId"] = pd.to_numeric(
            draft["collegeAthleteId"], errors="coerce"
        )

    # Create a mask for non-NaN values
    valid_ids_mask = ~draft["collegeAthleteId"].isna()

    # Only convert non-NaN values to int64
    if valid_ids_mask.any():
        draft.loc[valid_ids_mask, "collegeAthleteId"] = draft.loc[
            valid_ids_mask, "collegeAthleteId"
        ].astype("int64")

    # Step 2: Prepare columns for result DataFrame
    # Generate all stat column names
    stat_columns = [f"{cat}_{stat}" for _, (cat, stat) in stat_combinations.iterrows()]

    # Step 3: Initialize result DataFrame with all stats set to 0
    result = draft.copy()
    for col in stat_columns:
        result[col] = 0.0  # Initialize as float

    # Step 4: Create a more efficient way to look up stats - use groupby and pivot
    # This creates a DataFrame with playerId as index and all stat combinations as columns
    stat_df = pd.DataFrame()

    # Group by playerId and apply wide_to_long transformation for each category-statType
    for _, row in stat_combinations.iterrows():
        category = row["category"]
        stat_type = row["statType"]
        column_name = f"{category}_{stat_type}"

        # Create a temporary DataFrame with the specific stat
        temp_df = players[
            (players["category"] == category) & (players["statType"] == stat_type)
        ][["playerId", "stat"]].copy()

        # Rename 'stat' column to the specific stat name
        temp_df.rename(columns={"stat": column_name}, inplace=True)

        # First time, create the stat_df; otherwise merge
        if stat_df.empty:
            stat_df = temp_df
        else:
            stat_df = pd.merge(stat_df, temp_df, on="playerId", how="outer")

    # Fill any NaN values with 0
    stat_df = stat_df.fillna(0)

    # Step 5: For better performance, merge the stats with the draft data at once
    # Create a dict for easy lookup by playerId
    stat_dict = stat_df.set_index("playerId").to_dict("index")

    # Step 6: Populate stats for each draft pick
    for idx, row in result.iterrows():
        college_id = row["collegeAthleteId"]

        # Skip if NaN
        if pd.isna(college_id):
            continue

        # Make sure college_id is an integer before lookup
        try:
            college_id_int = int(college_id)
            # Get stats for this player
            player_stats = stat_dict.get(college_id_int, {})

            # Update stats in result DataFrame
            for stat_column, value in player_stats.items():
                result.loc[idx, stat_column] = float(value)

        except (ValueError, TypeError):
            # If conversion fails, skip this row
            continue

    # Ensure all NaN values are converted to 0
    result.fillna(0, inplace=True)

    return result


# Usage example:
result_df = merge_player_draft_data(players, draft)

# Step 5: Add Remaining Players (Undrafted)

In [ ]:
def add_remaining_players(result_df, players_df):
    """
    Add players from players_df that are not already in result_df
    by putting their playerId into collegeAthleteId and setting overall to 0

    Parameters:
    result_df (DataFrame): Draft data with player statistics
    players_df (DataFrame): Original player statistics

    Returns:
    DataFrame: Combined data with all players
    """
    # Get unique player IDs from both dataframes
    result_player_ids = set(result_df["collegeAthleteId"].dropna().astype(int))
    all_player_ids = set(players_df["playerId"].unique())

    # Find player IDs that are in players_df but not in result_df
    missing_player_ids = all_player_ids - result_player_ids
    print(f"Found {len(missing_player_ids)} players to add")

    if not missing_player_ids:
        return result_df  # No players to add

    # Get the column structure from result_df
    result_columns = result_df.columns.tolist()

    # Get all stat columns
    stat_columns = [
        col
        for col in result_columns
        if "_" in col and col not in ["collegeId", "nflTeamId"]
    ]

    # Create a dictionary to hold stats for each missing player
    player_rows = []

    # Group players_df by playerId to get all stats for each player
    grouped_players = players_df.groupby("playerId")

    # For each missing player, create a new row
    for player_id in missing_player_ids:
        # Start with a row filled with NaN/0
        new_row = {col: 0 for col in result_columns}

        # Set collegeAthleteId to the player's ID
        new_row["collegeAthleteId"] = player_id

        # If this player exists in the grouped data
        if player_id in grouped_players.groups:
            player_data = grouped_players.get_group(player_id)

            # Add player info if available
            if "player" in player_data.columns:
                new_row["name"] = player_data["player"].iloc[0]

            if "team" in player_data.columns:
                new_row["collegeTeam"] = player_data["team"].iloc[0]

            if "conference" in player_data.columns:
                new_row["collegeConference"] = player_data["conference"].iloc[0]

            # Add stats for this player
            for _, row in player_data.iterrows():
                category = row.get("category", "")
                stat_type = row.get("statType", "")
                if category and stat_type:
                    col_name = f"{category}_{stat_type}"
                    if col_name in stat_columns:
                        new_row[col_name] = row.get("stat", 0)

        player_rows.append(new_row)

    # Create DataFrame from the new rows
    new_players_df = pd.DataFrame(player_rows)

    # Set column types to match result_df
    for col in result_columns:
        if col in new_players_df.columns and col in result_df.columns:
            new_players_df[col] = new_players_df[col].astype(result_df[col].dtype)

    # Combine with original result_df
    combined_df = pd.concat([result_df, new_players_df], ignore_index=True)

    return combined_df


# Usage:
final_df = add_remaining_players(result_df, players)

# Step 5: Export to Feather

In [ ]:
def prepare_for_feather_export(df):
    """
    Prepare a DataFrame for feather export by fixing type issues
    """
    # Make a copy to avoid modifying the original
    df_clean = df.copy()

    # Handle object columns - convert to string
    object_columns = df_clean.select_dtypes(include=["object"]).columns
    for col in object_columns:
        # Convert non-string objects to strings
        df_clean[col] = df_clean[col].astype(str)

    # Handle int64/float64 conversion for numeric columns
    numeric_columns = df_clean.select_dtypes(include=["int64", "float64"]).columns
    for col in numeric_columns:
        # Ensure numeric columns are float64 for consistency
        df_clean[col] = df_clean[col].astype("float64")

    # Fix any potential datetime columns
    datetime_columns = df_clean.select_dtypes(include=["datetime64"]).columns
    for col in datetime_columns:
        # Convert to string if needed
        df_clean[col] = df_clean[col].dt.strftime("%Y-%m-%d %H:%M:%S")

    return df_clean


# Usage:
cleaned_df = prepare_for_feather_export(final_df)
cleaned_df.to_feather("draft_data.feather")

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.info()